# Agenda: Mask Index

- Comparisons
- Broadcasts and comparisons
- Using that to filter our series with a "boolean index" or a "mask index"
- Complex comparisons with "and" and "or"

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [7]:
s = Series([10,20,30,40,50,60,70], index=list('abcdefg'))

In [9]:
s.loc['d']

40

In [10]:
s.iloc[4]

50

In [ ]:
#inside of the []